In [ ]:
import numpy as np # linear algebra
from tqdm import tqdm
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import scipy
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from tqdm import tqdm 
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS = 100

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix, coo_matrix, dok_matrix, vstack, hstack

%matplotlib inline 
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_union
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
import pickle
import string
from sklearn.preprocessing import normalize
import timeit

stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
remove_list =  ["The", "ANN", "A", "In", "two", "However", "life", "But", "MAL", 
                "AniDB", "named", "He", "find", "[Written", "Rewrite]", "become", "must", "this", "also", 
                "when", "as", "get", "ann", "no", "it", "It", "AniDB", "s"] #hand typing
remove_list2 =  ["amount", "animes", "s", "no", "AniDB", "Source", "written mal rewrite", "mal", "Rewrite", "written"]

In [ ]:
""" i scrapped description of each anime for content recommender"""
anime_path = "../input/anime-description/AnimeProcessed.csv"

def binning(col, cut_points, labels=None):
    #Define min and max values:
    minval = col.min()
    maxval = col.max()
    #create list by adding min and max to cut_points
    break_points = [minval] + cut_points + [maxval]
    #if no labels provided, use default labels 0 ... (n-1)
    if not labels:
        labels = range(len(cut_points)+1)
    #Binning using cut function of pandas
    colBin = pd.cut(col, bins=break_points, labels=labels, include_lowest=True)
    return colBin

def create_soup(x):
    return ''.join(x['desc']) + "\n" +  ''.join(x["overall_rating"]) + "\n" + ''.join(x['name']) + "\n" +''.join(x['genre'])  + "\n" +''.join(x['type']) +"\n" + ''.join(x["episodes"]) +"\n" + ''.join(x["members"])

def load_anime(path) :
    anime = pd.read_csv(anime_path, sep=",")
    anime.rename(columns = {'animeid':'anime_id', "rating" : "overall_rating"}, inplace = True)
    anime['desc'] =anime['desc'].map(str).apply(lambda x: " ".join(x for x in x.split() if x not in (stop_words, remove_list)))
    
    anime["members"] = binning(anime["members"], [200000, 400000 ,600000],
                               ["seasonal", "normal", "popular", "mainstream"])
    
    anime["episodes"] = binning(anime["episodes"], [13, 24 ,52],
                                ["short animes", "normal animes", "long animes", "mainstream animes"])
    
    anime["overall_rating"] = binning(anime["overall_rating"], [3, 5 ,7],
                                ["bad", "neutral", "cool", "perfect"])
    
    for col in ["desc", "name", "genre", "type", "members", "episodes", "year", "overall_rating"] :
        anime[col] = anime[col].map(str)
        
    anime['soup'] = anime.apply(create_soup, axis =1)
    anime['soup'] = anime["soup"].apply(lambda x : re.sub('[^A-Za-z]+', ' ', str(x)))
    anime['soup'] =anime['soup'].map(str).apply(lambda x: " ".join(x for x in x.split() if x not in remove_list2))
    return anime

anime = load_anime(anime_path)
ALL_ANIMES = set(anime['anime_id'])
print("Shape of anime dataframe is {0}".format(anime.shape))
anime.head()

In [ ]:
rating_path = "../input/anime-recommendations-database/rating.csv"

def user_rating_matrix():
    URM = pd.read_csv(rating_path, sep=",")
    URM.rating.replace({-1: np.nan}, regex=True, inplace = True)
    URM.dropna(inplace = True)
    animes = URM.anime_id.unique()
    users = URM.user_id.unique()
    n_animes = len(animes)
    n_users =  len(users)
    print("Shape of URM dataframe is {0}".format(URM.shape))
    print("There's {0} users and {1} animes".format(str(n_users), str(n_animes)))
    return URM

URM = user_rating_matrix()
URM.head()

In [ ]:
def merge_it_all(df1, df2, n_sample) :
    df = pd.merge(df1, df2, left_on ="anime_id", right_on = "anime_id")
    df = df[df["user_id"] <= n_sample]
    return df

df = merge_it_all(anime, URM, 6000)

def get_unbiased_rating(df) : 
    users_interactions_count_df = df.groupby(['user_id', 'anime_id']).size().groupby('user_id').size()
    print('# users: %d' % len(users_interactions_count_df))
    users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 8].reset_index()[['user_id']]
    print('# users with at least 8 interactions: %d' % len(users_with_enough_interactions_df))
    print('# of interactions: %d' % len(df))
    interactions_from_selected_users_df = df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
    print('# of interactions from users with at least 8 interactions: %d' % len(interactions_from_selected_users_df))
    popularity = interactions_from_selected_users_df.groupby('anime_id').size().reset_index(name='popularity')
    interactions_full_df = pd.merge(popularity, interactions_from_selected_users_df)
    interactions_full_df.rename({"rating" : "user_rating"}, inplace = True, axis = 1)
    return interactions_full_df, interactions_from_selected_users_df

interactions_full_df, interactions_from_selected_users_df = get_unbiased_rating(df)
interactions_full_df.head(5)

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [ ]:
ALL_ANIMES = set(anime['anime_id'])

def get_anime_watched(user_id, interactions_df):
    try:
        anime_rated = interactions_df.loc[user_id]['anime_id']
        return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
    except KeyError:
        return []
    
def get_not_watched_anime_sample(user_id, sample_size, seed=42):
        anime_watched = get_anime_watched(user_id, interactions_full_indexed_df)
        non_watched_anime = ALL_ANIMES - anime_watched
        non_watched_anime_sample = random.sample(non_watched_anime, sample_size)
        return set(non_watched_anime_sample)

def _verify_hit_top_n(anime_id, recommended_anime, topn):        
        try:
            index = next(i for i, c in enumerate(recommended_anime) if c == anime_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index
    
def evaluate_model_for_anime_watcher(model, user_id):
    
    anime_watched_values_testset = interactions_test_indexed_df.loc[user_id]
    if type(anime_watched_values_testset['anime_id']) == pd.Series:
        user_anime_watched_testset = set(anime_watched_values_testset['anime_id'])
    else:
        user_anime_watched_testset = set([anime_watched_values_testset['anime_id']])  
    user_anime_watched_count_testset = len(user_anime_watched_testset) 

    #Getting a ranked recommendation list from a model for a given donor
    user_recs_df = model.recommend_anime(user_id,
                                         anime_to_ignore=get_anime_watched(
                                             user_id,
                                             interactions_train_indexed_df),
                                         topn=100000000)
    
    hits_at_3_count = 0
    hits_at_5_count = 0
    hits_at_10_count = 0
    #For each anime seen by user 
    for anime_id in user_anime_watched_testset:
        #Getting a random sample (100) non watched anime 
        non_watched_anime_sample = get_not_watched_anime_sample(user_id,
                                                            sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS,
                                                            seed=42)
        #Combining the current anime with the 100 random anime
        anime_to_filter_recs = non_watched_anime_sample.union(set([anime_id]))
        #Filtering only recommendations that are either the donated project or from a random sample of 100 non-donated projects
        valid_recs_df = user_recs_df[user_recs_df['anime_id'].isin(anime_to_filter_recs)]                    
        valid_recs = valid_recs_df['anime_id'].values
        
        #Verifying if the current anime is among the Top-N recommended projects
        hit_at_3, index_at_3 = _verify_hit_top_n(anime_id, valid_recs, 3)
        hits_at_3_count += hit_at_3
        hit_at_5, index_at_5 = _verify_hit_top_n(anime_id, valid_recs, 5)
        hits_at_5_count += hit_at_5
        hit_at_10, index_at_10 = _verify_hit_top_n(anime_id, valid_recs, 10)
        hits_at_10_count += hit_at_10
        
    recall_at_3 = hits_at_3_count / float(user_anime_watched_count_testset)
    recall_at_5 = hits_at_5_count / float(user_anime_watched_count_testset)
    recall_at_10 = hits_at_10_count / float(user_anime_watched_count_testset)

    anime_metrics = {'hits@3_count':hits_at_3_count, 
                     'hits@5_count':hits_at_5_count, 
                      'hits@10_count':hits_at_10_count, 
                      'watched_count': user_anime_watched_count_testset,
                      'recall@3': recall_at_3,
                      'recall@5': recall_at_5,
                      'recall@10': recall_at_10}
    return anime_metrics

In [ ]:
def evaluate_model(model):
    v = tqdm(enumerate(list(interactions_test_indexed_df.index.unique().values)))
    anime_user_metrics = [{**evaluate_model_for_anime_watcher(model, user_id), '_user_id': user_id} 
                          for idx, user_id in v]
    
    detailed_results_df = pd.DataFrame(anime_user_metrics) \
                        .sort_values('watched_count', ascending=False)
        
    global_recall_at_3 = detailed_results_df['hits@3_count'].sum() / float(detailed_results_df['watched_count'].sum())
    global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['watched_count'].sum())
    global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['watched_count'].sum())

    global_metrics = {'modelName': model.get_model_name(),
                      'recall@3': global_recall_at_3,
                      'recall@5': global_recall_at_5,
                      'recall@10': global_recall_at_10}   

    return global_metrics, detailed_results_df

# Popularity 

In [ ]:
item_popularity_df = interactions_full_df.groupby('anime_id')['user_rating'].sum().sort_values(ascending=False).reset_index()

class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, anime_df=None):
        self.popularity_df = popularity_df
        self.anime_df = anime_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_seen(self, user_id, anime_df):
        try:
            anime_rated = self.anime_df.loc[self.anime_df["user_id"] == user_id]['anime_id']
            return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
        except KeyError:
            return []
        
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['anime_id'].isin(self.get_seen(user_id, self.anime_df))] \
                               .sort_values('user_rating', ascending = False) \
                               .head(topn)

        if verbose:
            if self.anime_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.anime_df, how = 'left', 
                                                          left_on = 'anime_id', 
                                                          right_on = 'anime_id')[['rating', 
                                                                                  'anime_id',
                                                                                  'name', 
                                                                                  'type', 
                                                                                  'genre']]
        return recommendations_df

popularity_model = PopularityRecommender(item_popularity_df, anime)

In [ ]:
"""print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)"""

# content based

In [ ]:
anime_ids = anime['anime_id'].tolist()

class Users_Profiler : 
    
    def __init__(self, connector_matrix, anime_ids):
        self.connector_matrix = connector_matrix #base for cosine 
        self.anime_ids = anime_ids
        
    def get_anime_profile(self, anime_id):
        idx = self.anime_ids.index(anime_id)
        anime_profile = self.connector_matrix[idx:idx+1]
        return anime_profile
    
    def get_item_profiles(self, ids):
        item_profiles_list = [self.get_anime_profile(x) for x in np.ravel([ids])]
        item_profiles = vstack(item_profiles_list)
        return item_profiles
    
    def build_users_profile(self, user_id, interactions_indexed_df):
        interactions_anime_df = interactions_indexed_df.loc[user_id]
        user_item_profiles = self.get_item_profiles(interactions_anime_df['anime_id'])
        user_item_strengths = np.array(interactions_anime_df['user_rating']).reshape(-1,1) 
        #Weighted average of item profiles by the interactions strength
        user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths),
                                                  axis=0) / (np.sum(user_item_strengths)+1) #+1 added

        user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
        return user_profile_norm
    
    def build_users_profiles(self): 
        interactions_indexed_df = interactions_full_df[interactions_full_df['anime_id'].isin(anime['anime_id'])].set_index('user_id')
        user_profiles = {}
        for user_id in tqdm(interactions_indexed_df.index.unique()):
            user_profiles[user_id] = self.build_users_profile(user_id, interactions_indexed_df)
        return user_profiles

In [ ]:
""" Initialize vectorizer"""
    
word_vectorizer = TfidfVectorizer(ngram_range =(1,4),
                             min_df=5, max_df=0.9,
                             strip_accents='unicode',
                             stop_words = 'english',
                             analyzer = 'word',
                             use_idf=1,
                             smooth_idf=1,
                             sublinear_tf=1)

tfidf_matrix = word_vectorizer.fit_transform(anime['soup'])
tfidf_feature_names = word_vectorizer.get_feature_names()

""" compute users profiles """
U2P = Users_Profiler(tfidf_matrix, anime_ids)
U2tfifd = U2P.build_users_profiles()

In [ ]:
class BaseRecommender:
    
    MODEL_NAME = 'BaseforCosine'
    
    def __init__(self, users_profile, connector_matrix, anime_df=None):
        self.anime_ids = anime_ids
        self.anime_df = anime_df
        self.users_profile=  users_profile
        self.connector_matrix = connector_matrix
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_anime_to_users_profile(self, user_id, topn=1000):
        #Computes the cosine similarity between the donor profile and all project profiles
        cosine_similarities = cosine_similarity(self.users_profile[user_id], self.connector_matrix)
        #Gets the top similar projects
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar projects by similarity
        similar_anime = sorted([(anime_ids[i], cosine_similarities[0,i]) for i in similar_indices],
                                  key=lambda x: -x[1])
        return similar_anime
    
    def get_seen(self, user_id, anime_df):
        try:
            anime_rated = self.anime_df.loc[self.anime_df["user_id"] == user_id]['anime_id']
            return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
        except KeyError:
            return []
        
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10, verbose=False):
        similar_anime = self._get_similar_anime_to_users_profile(user_id)
        #Ignores projects the donor has already donated
        similar_anime_filtered = list(filter(lambda x: x[0] not in self.get_seen(user_id, 
                                                                            self.anime_df), 
                                             similar_anime))
        
        recommendations_df = pd.DataFrame(similar_anime_filtered, columns=['anime_id', 'user_rating']).head(topn)

        recommendations_df = recommendations_df.merge(self.anime_df, how = 'left', 
                                                    left_on = 'anime_id', 
                                                    right_on = 'anime_id')[['anime_id', 'user_rating', 'name', 'genre']].drop_duplicates()


        return recommendations_df

ContentBasedmodel = BaseRecommender(U2tfifd, tfidf_matrix, anime)

In [ ]:
ContentBasedmodel.recommend_anime(8)

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = evaluate_model(ContentBasedmodel)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df = cb_detailed_results_df[['_user_id', 'watched_count', "hits@3_count", 'hits@5_count','hits@10_count', 
                                                'recall@3','recall@5','recall@10']]
cb_detailed_results_df.head(10)

# WORD EMBEDDING

In [ ]:
def generate_doc_vectors(s):
    words = str(s).lower().split() 
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        if w in embeddings_index:
            M.append(embeddings_index[w])
    v = np.array(M).sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

EMBEDDING_FILE = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

embeddings_index = {}
f = open(EMBEDDING_FILE, encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
        
xtrain = anime["soup"].values
xtrain_embeddings = [generate_doc_vectors(x) for x in tqdm(xtrain)]
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [ ]:
xtrain_embeddings = sc.fit_transform(xtrain_embeddings)
xtrain_embeddings = csr_matrix(xtrain_embeddings)
U2f = Users_Profiler(xtrain_embeddings, anime_ids)
U2fastt = U2f.build_users_profiles()
fastcontent = BaseRecommender(U2fastt, xtrain_embeddings, anime)

In [ ]:
print('Evaluating Content-Based Filtering model...')
fglobal_metrics, fdetailed_results_df = evaluate_model(fastcontent)
print('\nGlobal metrics:\n%s' % fglobal_metrics)
fdetailed_results_df = fdetailed_results_df[['_user_id', 'watched_count', "hits@3_count", 'hits@5_count','hits@10_count', 
                                                'recall@3','recall@5','recall@10']]
fdetailed_results_df.head(10)

pretrained word embedding matrix seems not very efficient for content recommender compared to tfifd

## KNN based 

In [ ]:
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),
                            pd.get_dummies(anime[["type"]]),
                            pd.get_dummies(anime[["episodes"]]),
                            pd.get_dummies(anime[["members"]]),
                            pd.get_dummies(anime[["overall_rating"]])], axis=1)

anime_features.head()

In [ ]:
anime_ids = anime["anime_id"].tolist()
from sklearn.preprocessing import MaxAbsScaler
mabs = MaxAbsScaler()
anime_features_scale = mabs.fit_transform(anime_features)
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='brute', metric = "cosine").fit(anime_features_scale)
distances, indices = nbrs.kneighbors(anime_features_scale)
from scipy.sparse import csr_matrix
indices =  csr_matrix(indices)
distances =  csr_matrix(distances)

U2KNN = Users_Profiler(distances, anime_ids)
KNN_model = U2KNN.build_users_profiles()

In [ ]:
KNN_RECO = BaseRecommender(KNN_model, distances, anime)
print('Evaluating Content-Based Filtering model...')
global_metrics, detailed_results_df = evaluate_model(KNN_RECO)
print('\nGlobal metrics:\n%s' % global_metrics)

# MF

In [ ]:
pivpiv = interactions_full_df.pivot(index = 'user_id', columns ='anime_id', values = 'user_rating')
R= np.array(interactions_full_df.pivot(index = 'user_id', columns ='anime_id', values = 'user_rating').fillna(0))
iddd = pivpiv.index.values 

In [ ]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

        """ R – The user-anime rating matrix
            K – Number of latent features
            alpha – Learning rate for stochastic gradient descent
            beta – Regularization parameter for bias
            iterations – Number of iterations to perform stochastic gradient descent"""
    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in tqdm(range(self.iterations)):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 5 == 0:
                print("Iteration: %d ; mse = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [ ]:
mf = MF(R, K=200, alpha=0.009999, beta=0.01, iterations=120)
training_process = mf.train()
pred_df = mf.full_matrix()
cf_predictions_df = pd.DataFrame(pred_df, columns = pivpiv.columns, index=iddd).transpose()

In [ ]:
class CF_SG_Recommender:
    
    MODEL_NAME = 'Collaborative Filtering With Gradiant Descent'
    
    def __init__(self, cf_predictions_df, anime_df=None): #anime_df = keep_full_info
        self.cf_predictions_df = cf_predictions_df
        self.anime_df = anime_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_seen(self, user_id, anime_df):
        try:
            anime_rated = self.anime_df.loc[self.anime_df["user_id"] == user_id]['anime_id']
            return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
        except KeyError:
            return []
        
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10):
        # Get and sort the donor's predictions
        sorted_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted projects that the donor hasn't donated to
        recommendations_df = sorted_predictions[~sorted_predictions['anime_id'].isin(
            self.get_seen(user_id, self.anime_df))] \
        .sort_values('recStrength', ascending = False) \
        .head(topn)
                
        recommendations_df = recommendations_df.merge(self.anime_df, how = 'left', 
                                                      left_on = 'anime_id', 
                                                      right_on = "anime_id")[['recStrength',
                                                                              'anime_id',]].drop_duplicates()

        return recommendations_df
    
cfrsgd_model = CF_SG_Recommender(cf_predictions_df, interactions_full_df)
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cfrsgd_global_metrics, cfrsgd_detailed_results_df = evaluate_model(cfrsgd_model)
print('\nGlobal metrics:\n%s' % cfrsgd_global_metrics)
cfrsgd_detailed_results_df.head(10)

# SVD

In [ ]:
U, sigma, Vt = svds(R, k= 450)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
cf_preds_df = pd.DataFrame(predicted_ratings, 
                           columns = pivpiv.columns, 
                           index=iddd).transpose()

from math import sqrt 

print("RMSE is {0}".format(sqrt(mean_squared_error(R, predicted_ratings))))

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering '
    
    def __init__(self, cf_predictions_df, anime_df=None): #anime_df = keep_full_info
        self.cf_predictions_df = cf_predictions_df
        self.anime_df = anime_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_seen(self, user_id, anime_df):
        try:
            anime_rated = self.anime_df.loc[self.anime_df["user_id"] == user_id]['anime_id']
            return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
        except KeyError:
            return []
        
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10):
        # Get and sort the donor's predictions
        sorted_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted projects that the donor hasn't donated to
        recommendations_df = sorted_predictions[~sorted_predictions['anime_id'].isin(self.get_seen(user_id, self.anime_df))] \
        .sort_values('recStrength', ascending = False) \
        .head(topn)
                
        recommendations_df = recommendations_df.merge(self.anime_df, how = 'left', 
                                                      left_on = 'anime_id', 
                                                      right_on = "anime_id")[['recStrength',
                                                                              'anime_id', 'name']].drop_duplicates()

        return recommendations_df
    
cf_model = CFRecommender(cf_preds_df, anime)

# Hybrid 

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_model, anime_df):
        self.cb_rec_model = cb_rec_model
        self.cf_model = cf_model
        self.anime_df = anime_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend_anime(user_id,topn=1000)\
        .rename(columns={'user_rating': 'recStrengthCB'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_model.recommend_anime(user_id, topn=1000)\
        .rename(columns={'recStrength': 'recStrengthCF'})
        
        #Combining the results by Project ID
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'inner', 
                                   left_on = 'anime_id', 
                                   right_on = 'anime_id')
        
        #Computing a hybrid recommendation score based on CF and CB scores
        recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF']
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('recStrengthHybrid', ascending=False).head(topn)

        recommendations_df = recommendations_df.merge(self.anime_df, how = 'left', 
                                                    left_on = 'anime_id', 
                                                    right_on = 'anime_id')[['recStrengthHybrid',
                                                                            'anime_id',
                                                                            "name"]]


        return recommendations_df
    
hybrid_model = HybridRecommender(ContentBasedmodel, cf_model, anime)

In [ ]:
print('Evaluating hybrid_model...')
hybrid_global_metrics, hybrid_detailed_results_df = evaluate_model(hybrid_model)
print('\nGlobal metrics:\n%s' % hybrid_global_metrics)
hybrid_detailed_results_df.head(10)

# Deep Learning

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Embedding, Reshape, merge
from keras.layers import concatenate, dot
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

In [ ]:
users = interactions_full_df.user_id.unique()
animes = interactions_full_df.anime_id.unique()
n_users = len(users)
n_animes =  len(animes)

userid2idx = {o:i for i,o in enumerate(users)}
animesid2idx = {o:i for i,o in enumerate(animes)}
interactions_full_df['user_id'] = interactions_full_df['user_id'].apply(lambda x: userid2idx[x])
interactions_full_df['anime_id'] = interactions_full_df['anime_id'].apply(lambda x: animesid2idx[x])
#interactions_from_selected_users_df.rename(columns={"rating": "user_rating"}, inplace = True)

train, valid = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

In [ ]:
n_latent_factors_user = 150
n_latent_factors_animes = 500
n_latent_factors_mf = 3

""" Anime MultiLayer Perceptron """
animes_input = Input(shape=[1],name='Item')
animes_embedding_mlp = Embedding(n_animes + 1, 
                                 n_latent_factors_animes, 
                                 name='Animes-Embedding-MLP')(animes_input)
animes_vec_mlp = Flatten(name='FlattenAnimes-MLP')(animes_embedding_mlp)

""" Anime embedding for matrix factorisation """
animes_embedding_mf = Embedding(n_animes + 1,
                                n_latent_factors_mf,
                                name='Animes-Embedding-MF')(animes_input)
animes_vec_mf = Flatten(name='FlattenAnimes-MF')(animes_embedding_mf)

""" User MultiLayer Perceptron """

user_input = Input(shape=[1],name='User')
user_vec_mlp = Flatten(name='FlattenUsers-MLP')(Embedding(n_users + 1,
                                                          n_latent_factors_user,
                                                          name='User-Embedding-MLP')(user_input))
user_vec_mlp = Dropout(0.2)(user_vec_mlp)

""" Users embedding for matrix factorisation """
user_vec_mf = Flatten(name='FlattenUsers-MF')(Embedding(n_users + 1, 
                                                        n_latent_factors_mf,
                                                        name='User-Embedding-MF')(user_input))
user_vec_mf = Dropout(0.2)(user_vec_mf)

concat = concatenate([animes_vec_mlp, user_vec_mlp])
concat_dropout = Dropout(0.2)(concat)
dense = Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = BatchNormalization(name='Batch')(dense)
dense_2 = Dense(100,name='FullyConnected-1')(dense_batch)
dense_batch_2 = BatchNormalization(name='Batch-2')(dense_2)
dense_3 = Dense(50,name='FullyConnected-2')(dense_batch_2)
dense_4 = Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = dot([animes_vec_mf, user_vec_mf], axes = -1, name='Dot')
pred_mlp = Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = concatenate([pred_mf, pred_mlp],name='Concat-MF-MLP')
result_combine = Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = Dense(100,name='FullyConnected-4')(result_combine)

result = Dense(1,name='Prediction')(deep_combine)
DeepCF = Model([user_input, animes_input], result)
opt = Adam(lr =0.09)
DeepCF.compile(optimizer=opt ,loss= 'mean_absolute_error')
DeepCF.summary()

In [ ]:
history = DeepCF.fit([train.user_id, train.anime_id], train.user_rating, batch_size=512, epochs=5, 
          validation_data=([valid.user_id, valid.anime_id], 
            valid.user_rating))

In [ ]:
import math
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print ('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss)))

In [ ]:
user_map = {}
anime_map = {}

def getID(val, tag):
    return tags_dict[tag][val]

def mapping() : 
    print("start ...")
    unique_users = list(interactions_full_df['user_id'].value_counts().index)
    unique_animes = list(interactions_full_df['anime_id'].value_counts().index)
    for i, user in enumerate(unique_users):
        user_map[user] = i+1
    for i, anime in enumerate(unique_animes):
        anime_map[anime] = i+1
    tags_dict = {'users' : user_map, 'anime' : anime_map}
    print("mapping done ...")
    return tags_dict

tags_dict = mapping()

interactions_full_df['ani_id'] = interactions_full_df['user_id'].apply(lambda x : getID(x, 'users'))
interactions_full_df['user_idd'] = interactions_full_df['anime_id'].apply(lambda x : getID(x, 'anime'))

In [ ]:
class DeepRecommender:
    
    MODEL_NAME = 'PredictedRating_DeepRec'
    
    def __init__(self, deep_model, anime_users_df=None):
        self.deep_model = deep_model
        self.anime_users_df = anime_users_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_seen(self, user_id, anime_users_df):
        try:
            anime_rated = self.anime_users_df.loc[self.anime_users_df["user_id"] == user_id]['anime_id']
            return set(anime_rated if type(anime_rated) == pd.Series else [anime_rated])
        except KeyError:
            return []
    
    def rate(self, deep_model, user_id, anime_id):
        return deep_model.predict([np.array([user_id]), np.array([anime_id])])[0][0]
    
    def predict_rating(self, anime_id, userid):
        return self.rate(DeepCF, anime_id - 1, userid - 1)
    
    def build_rec_df(self, user_id) : 
        print("starting to build rec df ...")
        user_ratings = self.anime_users_df.loc[self.anime_users_df['ani_id'] == user_id][['user_idd', 'ani_id', 'user_rating']]
        user_ratings['predicted_amt'] = user_ratings.apply(lambda x: self.predict_rating(user_id, x['user_idd']), axis=1)
        recommendations = self.anime_users_df.loc[self.anime_users_df['user_idd'].isin(user_ratings['user_idd']) == False][['user_idd', "anime_id"]]\
        .drop_duplicates()
        recommendations['Predicted_ratings'] = recommendations.apply(lambda x: self.predict_rating(user_id, x['user_idd']), axis=1)
        print("done ...")
        return recommendations
    
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10, verbose=False):
        print("recommendations ...")
        recommendations =  self.build_rec_df(user_id)
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = recommendations[~recommendations['anime_id'].isin(self.get_seen(user_id, self.anime_users_df))] \
        .sort_values('Predicted_ratings', ascending = False) \
        .head(topn)

        if verbose:
            if self.anime_users_df is None:
                raise Exception('"anime_users_df" is required in verbose mode')

            recommendations_df = pd.merge(recommendations_df, self.anime_users_df, how = 'right', 
                                                          left_on = 'anime_id', 
                                                          right_on = 'anime_id').drop_duplicates()
        
        return recommendations_df
    
    
deep_colab_model= DeepRecommender(DeepCF, interactions_full_df)

I'll not trying to evaluate this model coz it tooks a while something like ~20h for 5k users the predict method is a bit slow.

# **Draw a try with a user with fews rated anime for easier comparison

In [ ]:
print(pop_rec.__len__(), cbr_rec.__len__(), cf_rec.__len__(), hybrid_rec.__len__(), embed_rec.__len__(), deep_colab_rec.__len__(), deep_rec.__len__(), mf_rec.__len__(), KNN_rec.__len__())

In [ ]:
def comparison(user, top_n) : 
    anime_list = get_anime_watched(user, interactions_full_df.set_index("user_id"))
    rated = anime[anime["anime_id"].isin(anime_list)]["name"].tolist()

    pop_rec = popularity_model.recommend_anime(user).head(top_n)
    pop_rec = pd.merge(anime, pop_rec, how =  "right", left_on = "anime_id",right_on = "anime_id")[["name"]]
    pop_rec = list(pop_rec.name)
    cbr_rec = ContentBasedmodel.recommend_anime(user).head(top_n)["name"].tolist()
    cf_rec = cf_model.recommend_anime(user).head(top_n)["name"].tolist()
    hybrid_rec = hybrid_model.recommend_anime(user).head(top_n)["name"].tolist()
    embed_rec = fastcontent.recommend_anime(user).head(top_n)["name"].tolist()
    deep_colab_rec = deep_colab_model.recommend_anime(user)
    deep_rec = pd.merge(anime, deep_colab_rec, how =  "right",  left_on = "anime_id",
                        right_on = "anime_id")[["name"]].fillna("None")
    deep_rec = list(deep_rec.name[:top_n])
    mf_rec = cfrsgd_model.recommend_anime(user).head(top_n)
    mf_rec = pd.merge(anime, mf_rec,how =  "right",  left_on = "anime_id",right_on = "anime_id")[["name"]]
    mf_rec = list(mf_rec.name)
    KNN_rec = KNN_RECO.recommend_anime(user).head(top_n)["name"].tolist()

    d = {'Popularity': pop_rec,
         'KNN-Based' : KNN_rec,
         'Deep_Content_based' : embed_rec,
         'Content-Based': cbr_rec,
         'Collaborative-Filtering': cf_rec,
         'Hybrid': hybrid_rec,
         'Deep Collaborative-Filtering' :  deep_rec, 
         "Matrix Factorization" : mf_rec}

    d = pd.DataFrame(data=d)
    return rated, d

rated, compare = comparison(25, 5)

print("User has seen ... :")
for i in rated :
    print(i)
    
print("Recommendations ... :")
compare

In [ ]:
rated, compare = comparison(8, 5)

print("User has seen ... :")
for i in rated :
    print(i)
    
print("Recommendations ... :")
compare

In [ ]:
rated, compare = comparison(55, 5)

print("User has seen ... :")
for i in rated :
    print(i)
    
print("Recommendations ... :")
compare

In [ ]:
rated, compare = comparison(5, 5)

print("User has seen ... :")
for i in rated :
    print(i)
    
print("Recommendations ... :")
compare

In [ ]:
rated, compare = comparison(28, 5)

print("User has seen ... :")
for i in rated :
    print(i)
    
print("Recommendations ... :")
compare

# Graph Based

In [ ]:
import networkx as nx

G =  nx.Graph() #create the empty graph
#users nodes
G.add_nodes_from(interactions_full_df["user_id"], bipartite="Users")
#animes nodes
G.add_nodes_from(interactions_full_df["anime_id"], bipartite="Animes")
#edge between users and animes
G.add_edges_from(zip(interactions_full_df["anime_id"], interactions_full_df["user_id"]))

""" bipartites projection """
users_nodes =  [n for n in G.nodes() if G.node[n]
                ["bipartite"] ==  "Users"]
animes_nodes = [n for n in G.nodes() if G.node[n]
                ["bipartite"] ==  "Animes"]
UsersG = nx.bipartite.projected_graph(G, nodes = users_nodes)
AnimeG =  nx.bipartite.projected_graph(G, nodes = animes_nodes)

In [ ]:
def shared_partition_nodes(G, node1, node2):
    # Check that the nodes belong to the same partition
    #assert G.node[node1]['bipartite'] == G.node[node2]['bipartite']

    # Get neighbors of node 1: nbrs1
    nbrs1 = G.neighbors(node1)
    # Get neighbors of node 2: nbrs2
    nbrs2 = G.neighbors(node2)

    # Compute the overlap using set intersections
    overlap = set(nbrs1).intersection(nbrs2)
    return overlap

# Print the number of shared repositories between users 'u7909' and 'u2148'
print(len(shared_partition_nodes(G, 38, 20)))

In [ ]:
# Define get_nodes_from_partition()
def get_nodes_from_partition(G, partition):
    # Initialize an empty list for nodes to be returned
    nodes = []
    # Iterate over each node in the graph G
    for n in G.nodes():
        # Check that the node belongs to the particular partition
        if G.node[n]['bipartite'] == partition:
            # If so, append it to the list of nodes
            nodes.append(n)
    return nodes

In [ ]:
def anime_similarity(G, anime1, anime2, users_proj_nodes) : 
    shared_nodes = shared_partition_nodes(G, anime1, anime2)
    return len(shared_nodes) / len(users_proj_nodes)


def user_similarity(G, user1, user2, proj_nodes):
    shared_nodes = shared_partition_nodes(G, user1, user2)
    return len(shared_nodes) / len(proj_nodes)

# Compute the similarity score between users 'u4560' and 'u1880'
animes_nodes = get_nodes_from_partition(G, 'Animes')
users_nodes = get_nodes_from_partition(G, 'Users')
similarity_score = user_similarity(G, 38, 20, animes_nodes)
similarity_score_anime = anime_similarity(G, 1, 6, users_nodes)
print(similarity_score)
print("similarity between trigun and cowboy bebop is {0}".format(similarity_score_anime))

In [ ]:
anime_name =anime[["name", "anime_id"]]
top_n_dict = {}
def most_similar_to(anime_id) : 
    #compute similarity dict
    for others_animes in interactions_full_df["anime_id"].unique() : 
        top_n_dict.update({others_animes : anime_similarity(G, anime_id, others_animes, users_nodes)})
        
    sorted_dict = [(key, value) for (key, value) in sorted(top_n_dict.items(), reverse = False)][1:15]
    dtf = pd.DataFrame(sorted_dict).rename(columns={0:"anime_id", 1:"most_similar_score"})
    merged_topn = dtf.merge(anime_name, left_on = "anime_id", right_on = "anime_id")\
    .sort_values(by="most_similar_score", ascending = False)
    return merged_topn
    
merged_topn = most_similar_to(38)
merged_topn

In [ ]:
from collections import defaultdict

def most_similar_users(G, user, user_nodes, proj_nodes):
    # Data checks
    assert G.node[user]['bipartite'] == 'Users'

    # Get other nodes from user partition
    user_nodes = set(user_nodes) 
    user_nodes.remove(user)

    # Create the dictionary: similarities
    similarities = defaultdict(list)
    for n in user_nodes:
        similarity = user_similarity(G, user, n, proj_nodes)
        similarities[similarity].append(n)

    # Compute maximum similarity score: max_similarity
    max_similarity = max(similarities.keys())

    # Return list of users that share maximal similarity
    return similarities[max_similarity]

user_nodes = get_nodes_from_partition(G, 'Users')
animes_nodes = get_nodes_from_partition(G, 'Animes')

print(most_similar_users(G, 38, user_nodes, animes_nodes))

In [ ]:
class NetworkRecommender : 
        
    """ steps of a functional recommender :
    step 1 : return 10 users most similar to our 
    step 2 : look every anime watched by similar users 
    step 3 : substract anime not seen by our users in others users list 
    step 4 : compute most similar anime between our users watched list and others users watched list
    step 5 : return top_n"""
    
    MODEL_NAME = 'Network Recommender'
    
    def __init__(self, graph, animes_nodes, users_nodes) :
        self.graph = graph
        self.animes_nodes = animes_nodes
        self.users_nodes = users_nodes
    
    def get_model_name(self):
        return self.MODEL_NAME
    
    def anime_similarity(self, graph, anime1, anime2, users_nodes) : 
        shared_nodes = shared_partition_nodes(self.graph, anime1, anime2)
        return len(shared_nodes) / len(users_nodes)

    def user_similarity(self, graph, user1, user2, animes_nodes):
        shared_nodes = shared_partition_nodes(self.graph, user1, user2)
        return len(shared_nodes) / len(animes_nodes)
    
    def most_similar_users(self, sgraph, user_id, user_nodes, sanimes_nodes):
        # Get other nodes from user partition
        user_nodes = set(self.user_nodes) 
        user_nodes.remove(user_id)
        # Create the dictionary: similarities
        similarities = defaultdict(list)
        for n in user_nodes:
            similarity = user_similarity(self.graph, user_id, n, self.animes_nodes)
            similarities[similarity].append(n)
        # Compute maximum similarity score: max_similarity
        max_similarity = max(similarities.keys())
        # Return list of users that share maximal similarity
        return similarities[max_similarity]
    
    def most_similar(self, anime_id) : 
        for others_animes in interactions_full_df["anime_id"].unique() : 
            top_n_dict.update({others_animes : self.anime_similarity(self.graph, anime_id, 
                                                                 others_animes, self.users_nodes)})
        #sort_it and reconstruct in a dataframe
        sorted_dict = [(key, value) for (key, value) in sorted(top_n_dict.items(), reverse = False)][1:15]
        dtf = pd.DataFrame(sorted_dict).rename(columns={0:"anime_id", 1:"most_similar_score"})
        merged_topn = dtf.merge(anime_name, left_on = "anime_id", right_on = "anime_id")\
        .sort_values(by="most_similar_score", ascending = False)
        return merged_topn
    
    def recommend_anime(self, user_id, anime_to_ignore=[], topn=10, verbose=False):
        merged_topn = self.most_similar(anime_id)
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['anime_id'].isin(anime_to_ignore)] \
                               .sort_values('user_rating', ascending = False) \
                               .head(topn)
    
NxRec= NetworkRecommender(G, animes_nodes, users_nodes)
NxRec.recommend_anime(38)

In [ ]:
def recommend_repositories(G, from_user, to_user):
    # Get the set of repositories that from_user has contributed to
    from_repos = set(G.neighbors(from_user))
    # Get the set of repositories that to_user has contributed to
    to_repos = set(G.neighbors(to_user))

    # Identify repositories that the from_user is connected to that the to_user is not connected to
    return from_repos.difference(to_repos)

# Print the repositories to be recommended
print(recommend_repositories(G, 38, 20))